In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
import copy
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
import sys
#sys.path.insert(0, '../../')
sys.path.insert(0, '/home/luka/Developement/accetuation/')
from prepare_data import *

%run ../../prepare_data.py

In [125]:
data = Data('sl', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [3]:
data = Data('s', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [3]:
data = Data('l', accent_classification=True)
data.generate_data('letters_accent_classification_train',
                   'letters_accent_classification_test',
                   'letters_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [17]:
data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False, accent_classification=True)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
content = data._read_content('../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [29]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 5
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

#x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x_conv)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [30]:
history = model.fit_generator(data.generator('train', batch_size, content_location='../../data/'), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size, content_location='../../data/'), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2)

Epoch 1/100
18s - loss: 0.2440 - actual_accuracy: 0.0127 - val_loss: 0.1997 - val_actual_accuracy: 0.0196
Epoch 2/100
20s - loss: 0.1936 - actual_accuracy: 0.1401 - val_loss: 0.1682 - val_actual_accuracy: 0.2803
Epoch 3/100
21s - loss: 0.1673 - actual_accuracy: 0.3066 - val_loss: 0.1557 - val_actual_accuracy: 0.4210
Epoch 4/100
21s - loss: 0.1478 - actual_accuracy: 0.4039 - val_loss: 0.1361 - val_actual_accuracy: 0.5157
Epoch 5/100
20s - loss: 0.1352 - actual_accuracy: 0.4772 - val_loss: 0.1317 - val_actual_accuracy: 0.5081
Epoch 6/100
21s - loss: 0.1270 - actual_accuracy: 0.5089 - val_loss: 0.1187 - val_actual_accuracy: 0.5736
Epoch 7/100
20s - loss: 0.1190 - actual_accuracy: 0.5420 - val_loss: 0.1156 - val_actual_accuracy: 0.6108
Epoch 8/100
38s - loss: 0.1121 - actual_accuracy: 0.5818 - val_loss: 0.1050 - val_actual_accuracy: 0.6702
Epoch 9/100
38s - loss: 0.1072 - actual_accuracy: 0.6091 - val_loss: 0.1060 - val_actual_accuracy: 0.6591
Epoch 10/100
34s - loss: 0.1022 - actual_accur

34s - loss: 0.0403 - actual_accuracy: 0.8162 - val_loss: 0.0967 - val_actual_accuracy: 0.7634
Epoch 79/100
34s - loss: 0.0389 - actual_accuracy: 0.8244 - val_loss: 0.0854 - val_actual_accuracy: 0.7696
Epoch 80/100
34s - loss: 0.0409 - actual_accuracy: 0.8149 - val_loss: 0.0784 - val_actual_accuracy: 0.7684
Epoch 81/100
34s - loss: 0.0400 - actual_accuracy: 0.8171 - val_loss: 0.0816 - val_actual_accuracy: 0.7688
Epoch 82/100
37s - loss: 0.0392 - actual_accuracy: 0.8178 - val_loss: 0.0903 - val_actual_accuracy: 0.7594
Epoch 83/100
34s - loss: 0.0392 - actual_accuracy: 0.8168 - val_loss: 0.0795 - val_actual_accuracy: 0.7788
Epoch 84/100
35s - loss: 0.0398 - actual_accuracy: 0.8171 - val_loss: 0.0858 - val_actual_accuracy: 0.7581
Epoch 85/100
33s - loss: 0.0385 - actual_accuracy: 0.8176 - val_loss: 0.0842 - val_actual_accuracy: 0.7673
Epoch 86/100
34s - loss: 0.0393 - actual_accuracy: 0.8142 - val_loss: 0.0878 - val_actual_accuracy: 0.7546
Epoch 87/100
34s - loss: 0.0396 - actual_accuracy:

In [13]:
#model.load_weights('../word_accetuation/cnn_dictionary/test_accent_type_with_other_features/40_epoch.h5')
model.load_weights('/home/luka/Developement/accetuation/cnn/word_accetuation/cnn_dictionary/test_accent_type_with_other_features/40_epoch.h5')


In [126]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 252)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [127]:
model.load_weights('syllabled_letters/v1_0/20_epoch.h5')
#model = load_model('syllabled_letters/v2_5_3/40_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

In [28]:
num_examples = len(data.x_train)  # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 5168)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [9]:
model.load_weights('syllables/v1_0/20_epoch.h5')

In [80]:
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20

In [5]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
conv_input_shape=(23, 36)

# syllabled letters
# conv_input_shape=(10, 5168)


# othr_input = (140, )
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
# x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [6]:
model.load_weights('letters/v2_1/20_epoch.h5')
# model = load_model('letters/v2_1/40_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

In [34]:
y_array = np.asarray(data.y_test)
accentuation_length = (y_array > 0).sum()
scores = model.evaluate_generator(data.generator('test', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                            accentuation_length/(batch_size))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


actual_accuracy: 77.70%


In [31]:
y_array = np.asarray(data.y_test)
accentuation_length = (y_array > 0).sum()
predictions = model.predict_generator(data.generator('test', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                                      accentuation_length/(batch_size))

In [19]:
accented_vowels = ['ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']
vowels = ['a', 'e', 'i', 'o', 'u', 'ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']

In [33]:
accuracy, real_accuracy, errors = data.test_type_accuracy(predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

94.29493490865332
93.9893964424296


In [140]:
is_letters = False
def test_accuracy(data, predictions, x, x_other_features, y, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=None):
    errors = []
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    num_of_correct_pred_words = 0
    accentuation_index = 0
    eye = np.eye(len(accented_vowels), dtype=int)
    for i in range(len(y)):
        correct_prediction = True
        if is_letters:
            decoded_x = data.decode_x(x[i], dictionary)
        else:
            decoded_x = data.decode_syllable_x(x[i], syllable_dictionary)
        wrong_word = decoded_x
        correct_word = decoded_x
        
        for j in range(len(y[i])):
            if y[i][j] > 0:
                # ERROR AS IT IS CALCULATED
                #arounded_predictions = np.around(predictions[accentuation_index]).astype(int)
                
                # MAX ELEMENT ONLY
                # arounded_predictions = np.zeros(len(predictions[accentuation_index]))
                # arounded_predictions[np.argmax(predictions[accentuation_index]).astype(int)] = 1
                
                # MAX ELEMENT AMONGT POSSIBLE ONES
                #if i == 313:
                #    print(decoded_x)
                stressed_letter = get_accentuated_letter(data, decoded_x, j, vowels, syllables=not is_letters)
                possible_places = np.zeros(len(predictions[accentuation_index]))
                if stressed_letter == 'r':
                    possible_places[0] = 1
                elif stressed_letter == 'a':
                    possible_places[1] = 1
                    possible_places[2] = 1
                elif stressed_letter == 'e':
                    possible_places[3] = 1
                    possible_places[4] = 1
                    possible_places[5] = 1
                elif stressed_letter == 'i':
                    possible_places[6] = 1
                    possible_places[7] = 1
                elif stressed_letter == 'o':
                    possible_places[8] = 1
                    possible_places[9] = 1
                    possible_places[10] = 1
                elif stressed_letter == 'u':
                    possible_places[11] = 1
                    possible_places[12] = 1
                possible_predictions = predictions[accentuation_index] * possible_places
                
                arounded_predictions = np.zeros(len(predictions[accentuation_index]),  dtype=int)
                arounded_predictions[np.argmax(possible_predictions).astype(int)] = 1
                
                wrong_word = assign_word_accentuation_type(data, wrong_word, j, arounded_predictions, vowels, accented_vowels, syllables=not is_letters, debug=i==313)
                correct_word = assign_word_accentuation_type(data, correct_word, j, eye[int(y[i][j])], vowels, accented_vowels, syllables=not is_letters, debug=i==313)

                
                if (eye[int(y[i][j])] == arounded_predictions).all():
                    num_of_correct_pred += 1
                else:
                    correct_prediction = False

                accentuation_index += 1
        
        if correct_prediction:
            num_of_correct_pred_words += 1
        else:
            if is_letters:
                errors.append([i,
                               decoded_x[::-1],
                               data.decode_x_other_features(feature_dictionary, [x_other_features[i]]),
                               wrong_word[::-1],
                               correct_word[::-1]
                              ])
            else:
                errors.append([i,
                               decoded_x,
                               data.decode_x_other_features(feature_dictionary, [x_other_features[i]]),
                               wrong_word,
                               correct_word
                              ])
    return (num_of_correct_pred / float(num_of_pred)) * 100, (num_of_correct_pred_words / float(len(y))) * 100, errors

#predictions = model.predict([data.x_validate, data.x_other_features_validate])
accuracy, real_accuracy, errors = test_accuracy(data, predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)

In [139]:
def get_accentuated_letter(data, word, location, vowels, syllables=False, debug=False):
    #print(location)
    vowel_index = 0
    word_list = list(word)
    if not syllables:
        word_list = list(word)
    else:
        word_list = list(word[::-1])
    for i in range(len(word_list)):
        if data._is_vowel(word_list, i, vowels):
            if location == vowel_index:
                return word_list[i]
            vowel_index += 1

In [138]:
def assign_word_accentuation_type(data, word, location, y, vowels, accented_vowels, syllables=False, debug=False):
    vowel_index = 0
    if not syllables:
        word_list = list(word)
    else:
        word_list = list(word[::-1])
    for i in range(len(word_list)):
        if data._is_vowel(word_list, i, vowels):
            if location == vowel_index:
                if len(np.where(y==1)[0]) == 1:
                    word_list[i] = accented_vowels[np.where(y==1)[0][0]]
            vowel_index += 1
    if not syllables:
        return ''.join(word_list)
    else:
        return ''.join(word_list[::-1])

In [154]:
#95.79732341465194
#95.57224634062008
#92.9019436239653
#92.52180260478697
#93.40152426795026
#93.04813861462216
print(accuracy)
print(real_accuracy)

96.44276702038435
96.25225709785241


In [162]:
errors[1000:]

[[10164, 'odnest', 'Vm-u----', 'odnést', 'odnëst'],
 [47858, 'odnesti', 'Vm-n----', 'odnésti', 'odněsti'],
 [1056, 'odvečen', 'Agpmsan', 'odvéčen', 'odvëčen'],
 [26443, 'odvečen', 'Agpmsnn', 'odvéčen', 'odvëčen'],
 [9446, 'odvečna', 'Agpmda-', 'odvéčna', 'odvëčna'],
 [13032, 'odvečna', 'Agpnpa-', 'odvéčna', 'odvëčna'],
 [14054, 'odvečna', 'Agpmdn-', 'odvéčna', 'odvëčna'],
 [27156, 'odvečna', 'Agpfsn-', 'odvéčna', 'odvëčna'],
 [33116, 'odvečna', 'Agpnpn-', 'odvéčna', 'odvëčna'],
 [15928, 'odvečne', 'Agpfpn-', 'odvéčne', 'odvëčne'],
 [24340, 'odvečne', 'Agpfpa-', 'odvéčne', 'odvëčne'],
 [25369, 'odvečne', 'Agpfsg-', 'odvéčne', 'odvëčne'],
 [43628, 'odvečne', 'Agpmpa-', 'odvéčne', 'odvëčne'],
 [28391, 'odvečnega', 'Agpmsg-', 'odvéčnega', 'odvëčnega'],
 [30894, 'odvečnega', 'Agpnsg-', 'odvéčnega', 'odvëčnega'],
 [36504, 'odvečnega', 'Agpmsa-', 'odvéčnega', 'odvëčnega'],
 [16336, 'odvečnem', 'Agpmsl-', 'odvéčnem', 'odvëčnem'],
 [39828, 'odvečnem', 'Agpnsl-', 'odvéčnem', 'odvëčnem'],
 [12455

In [30]:
errors = [[el[0], el[1][::-1], el[2], el[3][::-1], el[4][::-1]] for el in errors]

In [156]:
errors.sort(key=lambda x: x[1])

In [158]:
name = 'letters_accent_classification'
#name = 'syllables_accent_classification'
#name = 'syllabled_letters_accent_classification'

In [159]:
output = open(name + '_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open(name + '_predictions.pkl', 'wb')
pickle.dump(predictions, output)
output.close()

In [8]:
pickle_input = open('letters_accent_classification_errors.pkl', 'rb')
letters_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_errors.pkl', 'rb')
syllables_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_errors.pkl', 'rb')
syllabled_letters_accent_classification_errors = pickle.load(pickle_input)

In [9]:
pickle_input = open('letters_accent_classification_predictions.pkl', 'rb')
letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_predictions.pkl', 'rb')
syllables_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_predictions.pkl', 'rb')
syllabled_letters_accent_classification_predictions = pickle.load(pickle_input)

In [10]:
accuracy, real_accuracy, errors = data.test_type_accuracy(letters_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllables_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllables_accent_classification_predictions")
print(accuracy)
print(real_accuracy)
accuracy, real_accuracy, errors = data.test_type_accuracy(syllabled_letters_accent_classification_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print("syllabled_letters_accent_classification_predictions")
print(accuracy)
print(real_accuracy)

letters_accent_classification_predictions
96.44276702038435
96.25225709785241
syllables_accent_classification_predictions
95.79732341465194
95.57224634062008
syllabled_letters_accent_classification_predictions
96.58862998213179
96.40593184524953


In [14]:
def compare_outputs(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] in outputs2_indices]

def get_unique_errors(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] not in outputs2_indices]


#print(len(data.x_test))
#print(len(predictions))
print(len(letters_accent_classification_errors))
print(len(syllables_accent_classification_errors))
print(len(syllabled_letters_accent_classification_errors))
print(len(compare_outputs(letters_accent_classification_errors, syllables_accent_classification_errors)))
print(len(compare_outputs(letters_accent_classification_errors, syllabled_letters_accent_classification_errors)))
print(len(compare_outputs(syllables_accent_classification_errors, syllabled_letters_accent_classification_errors)))


print(1  - 7073.0/52058)
collective_errors = compare_outputs(letters_accent_classification_errors, compare_outputs(syllabled_letters_accent_classification_errors, syllables_accent_classification_errors))
print(len(collective_errors))

# print(letters_word_accetuation[:20])
# print(syllables_word_accetuation[:20])
# print(syllabled_letters_word_accetuation[:20])
print(collective_errors)

#unique_err = get_unique_errors(get_unique_errors(letters_accent_classification_errors, syllables_accent_classification_errors), syllabled_letters_accent_classification_errors)
#unique_err = get_unique_errors(get_unique_errors(syllables_accent_classification_errors, letters_accent_classification_errors), syllabled_letters_accent_classification_errors)
unique_err = get_unique_errors(get_unique_errors(syllabled_letters_accent_classification_errors, letters_accent_classification_errors), syllables_accent_classification_errors)

1951
2305
1871
875
1011
890
0.8641323139575089
599
[[19524, 'as', 'Ncmsn-', 'äs', 'ás'], [15290, 'bedast', 'Agpmsnn', 'bédast', 'bědast'], [43337, 'bedast', 'Agpmsan', 'bédast', 'bědast'], [257, 'bedasta', 'Agpmda-', 'bédasta', 'bědasta'], [3932, 'bedasta', 'Agpmdn-', 'bédasta', 'bědasta'], [23282, 'bedaste', 'Agpfpa-', 'bédaste', 'bědaste'], [29181, 'bedaste', 'Agpfsg-', 'bédaste', 'bědaste'], [38586, 'bedaste', 'Agpmpa-', 'bédaste', 'bědaste'], [46960, 'bedaste', 'Agpfpn-', 'bédaste', 'bědaste'], [4402, 'bedastem', 'Agpnsl-', 'bédastem', 'bědastem'], [21354, 'bedastem', 'Agpmsl-', 'bédastem', 'bědastem'], [21594, 'bedastih', 'Agpndg-', 'bédastih', 'bědastih'], [47357, 'bedastih', 'Agpmdg-', 'bédastih', 'bědastih'], [26461, 'bedastim', 'Agpmsi-', 'bédastim', 'bědastim'], [27117, 'bedastim', 'Agpnsi-', 'bédastim', 'bědastim'], [31494, 'bedastim', 'Agpmpd-', 'bédastim', 'bědastim'], [37396, 'bedastim', 'Agpnpd-', 'bédastim', 'bědastim'], [39351, 'bedastim', 'Agpfpd-', 'bédastim', 'bědas

In [12]:
ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllables_accent_classification_predictions, syllabled_letters_accent_classification_predictions ]), axis=0 )
accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

97.29059548554133
97.14549156709825


In [176]:
output = open('ensemble_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open('ensemble_predictions.pkl', 'wb')
pickle.dump(predictions, output)
output.close()

In [15]:
print(len(unique_err))
#collective_errors[:1000]
unique_err[:1000]

569


[[14677, 'alergičnih', 'Agpndg-', 'alérgičnih', 'alěrgičnih'],
 [23706, 'anket', 'Ncfpg-', 'ankět', 'ankét'],
 [49392, 'anket', 'Ncfdg-', 'ankět', 'ankét'],
 [32228, 'bežne', 'Agpmpa-', 'běžne', 'béžne'],
 [43777, 'bežne', 'Agpfpa-', 'běžne', 'béžne'],
 [47661, 'bežne', 'Agpfpn-', 'běžne', 'béžne'],
 [10195, 'bežnem', 'Agpnsl-', 'běžnem', 'béžnem'],
 [46867, 'bežnem', 'Agpmsl-', 'běžnem', 'béžnem'],
 [14835, 'bežni', 'Agpfdn-', 'běžni', 'béžni'],
 [17234, 'bežni', 'Agpndn-', 'běžni', 'béžni'],
 [23480, 'bežni', 'Agpmpn-', 'běžni', 'béžni'],
 [48038, 'bežni', 'Agpfda-', 'běžni', 'béžni'],
 [3218, 'bežnih', 'Agpfdg-', 'běžnih', 'béžnih'],
 [7755, 'bežnih', 'Agpfdl-', 'běžnih', 'béžnih'],
 [16357, 'bežnih', 'Agpndl-', 'běžnih', 'béžnih'],
 [17446, 'bežnih', 'Agpndg-', 'běžnih', 'béžnih'],
 [20978, 'bežnih', 'Agpmdg-', 'běžnih', 'béžnih'],
 [21527, 'bežnih', 'Agpfpg-', 'běžnih', 'béžnih'],
 [23626, 'bežnih', 'Agpmpl-', 'běžnih', 'béžnih'],
 [31598, 'bežnih', 'Agpfpl-', 'běžnih', 'béžnih'],